In [212]:
# класс точки на игровом поле с декартовыми координатами
class Dot:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"({self.x},{self.y})"

    def __eq__(self, _dot):
        return self.x == _dot.x and self.y ==_dot.y 

    
# exception
class PlayBoardException(Exception):
    pass

class PlayBoardOutException(PlayBoardException):
    print("Точка выходит за игровое поле!")

class PlayBoardBuzyException(PlayBoardException):
    print("В эту точку уже стреляли! Посторите ввод координат выстрела!")

class PlayBoardOccupiedException(PlayBoardException): #исключение невозможности установки корабля в данной точке
    pass
    
    
# класс корабля
class Ship:
    def __init__(self, xy, l, horiz):   # xy  - координаты точки на игровом поле; 
        self.xy = xy                    # l - размерность корабля, horiz - горизонтальность (True, False)
        self.l = l
        self.horiz = horiz
        self.lives = l

    @property                          #возвращает точки, принадлежащие кораблю
    def ship_dots(self):
        list = []
        for i in range(self.l):
            _x = self.xy.x
            _y = self.xy.y
            if self.horiz:
                _x += i
            else:
                _y += i
            list.append(Dot(_x, _y))
        return list
    

class Playboard:

    def __init__(self, size = 6, visible = True):
        self.size = size
        self.visible = visible
        
        self.matrix = [["O"]*size for _ in range(size)]

        self.ships = []
        self.buzy = []

        
    def dot_out(self, d):
        return not (0 <= d.x < self.size) or not (0 <= d.y < self.size)
    
    def add_ship(self, ship, visible = True):
        for d in ship.ship_dots:
            if self.dot_out(d) or d in self.buzy:
                raise PlayBoardOccupiedException()
                   

        for p in ship.ship_dots:
            if visible:
                self.matrix[p.y][p.x] = "■"
            self.buzy.append(Dot(p.x, p.y))
        self.ship_env(ship)
        self.ships.append(ship)
        

    def ship_env(self, ship, visible = False):
        env = [(1, 1), (1, 0), (1, -1),
               (0, -1), (-1, -1), (-1, 0),
               (-1, 1), (0, 1)] # точки вокруг корабля
        
        for d in ship.ship_dots:
            for ex, ey in env:
                cur = Dot(d.x + ex, d.y + ey)
                if not self.dot_out(cur) and cur not in self.buzy:
                    if visible:
                        self.matrix[cur.y][cur.x] = "."
                    self.buzy.append(cur)
                    
    def shut(self, d):             # проверка: попал выстрел в корабль?
        
        if self.dot_out(d):
            raise PlayBoardOutException()
                
        for ship in self.ships:
            
            for shd in ship.ship_dots:
                if d in self.buzy:
                    raise PlayBoardBuzyException()
                 
                self.matrix[d.y][d.x] = "X"
                if d == shd and ship.lives:
                    self.buzy.append(d)  
                    ship.lives -= 1
                    if ship.lives <= 0:
                        print("корабль потоплен!")
                        return True
                    print("Корабль ранен")
                    return True
        self.buzy.append(d)                        
        print("Мимо!")            
        return False

                    
                        
    def __str__(self):
        source = "   | 0 | 1 | 2 | 3 | 4 | 5 |"
        for i, s in enumerate(self.matrix):
            source += f"\n {i} | " + " | ".join(s) + " |"
        if not self.visible:
            source = source.replace("■","O")
        return source


Точка выходит за игровое поле!
В эту точку уже стреляли! Посторите ввод координат выстрела!


In [213]:
from random import randint

class Player:
    
    def __init__(self):
        pass

    def place_ships(self, size = 6, visible = True):
        ships_lenght = [3, 2, 2, 1, 1, 1, 1]
        board = Playboard(visible = True)
        attempt = 0 # количнство попыток расставить корабли
        for s in ships_lenght:
            while True:
                ship = Ship(Dot(randint(0, size), randint(0, size)), s, randint(0, 1))
                attempt +=1
                if attempt > 2000:
                    return False
                try:
                    board.add_ship(ship)
                    break
                except PlayBoardOccupiedException:
                    pass
        board.buzy =[]
        return board


In [214]:
d1 = Dot(1, 3)
s1 = Ship(d1, 3, True)
# print(s1.ship_dots)
# print(s1.shuted(Dot(1,2)))
# pb = Playboard(visible = True)
# pb.add_ship(s1)
# pb.ship_env(s1)
# print(pb)
# print()
b = Player().place_ships()
print(b)
# s2 = Ship(Dot(3, 4),2, False)
# pb.add_ship(s2)
# pb.ship_env(s2)
# print(pb)


   | 0 | 1 | 2 | 3 | 4 | 5 |
 0 | ■ | O | O | ■ | O | O |
 1 | O | O | O | ■ | O | ■ |
 2 | ■ | ■ | O | ■ | O | O |
 3 | O | O | O | O | O | O |
 4 | ■ | ■ | O | ■ | O | O |
 5 | O | O | O | O | O | ■ |


In [233]:
print(b.shut(Dot(0,2)))
print(b)


PlayBoardBuzyException: 